In [1]:
import cv2
import numpy as np
import os
# from sklearn.neighbors import KNeighborsClassifier
def distance(v1, v2):
	# Eucledian 
	return np.sqrt(((v1-v2)**2).sum())

def knn(train, test, k=5):
	dist = []
	
	for i in range(train.shape[0]):
		# Get the vector and label
		ix = train[i, :-1]
		iy = train[i, -1]
		# Compute the distance from test point
		d = distance(test, ix)
		dist.append([d, iy])
	# Sort based on distance and get top k
	dk = sorted(dist, key=lambda x: x[0])[:k]
	# Retrieve only the labels
	labels = np.array(dk)[:, -1]
	
	# Get frequencies of each label
	output = np.unique(labels, return_counts=True)
	# Find max frequency and corresponding label
	index = np.argmax(output[1])
	return output[0][index]
#########################################3

cap=cv2.VideoCapture(0)

face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip=0
dataset_path='data/'
face_data=[]
labels=[]      #y values of data ie labels of images
class_id=0
names={} #mapping b/w id-name

#Data-preparation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id]=fx[:-4]   #.npy is sliced
        print("loaded "+fx)                      
        data_item=np.load(dataset_path+fx)            #file is loaded at face_data
        face_data.append(data_item)
        
        #create label for the class
        target=class_id*np.ones((data_item.shape[0],)) #check
        class_id+=1
        labels.append(target)
face_dataset=np.concatenate(face_data,axis=0)               #x-axis
face_labels=np.concatenate(labels,axis=0).reshape((-1,1))                  #y-axis

print(face_dataset.shape)
print(face_labels.shape)
#concatenate to a single dataset as training dataset which is accepted by knn
trainset=np.concatenate((face_dataset,face_labels),axis=1)
print(trainset.shape)
# knn=KNeighborsClassifier(n_neighbors=5)
# knn.fit(trainset[:,:-1],trainset[:,-1])
#testing
while True:
    ret,frame=cap.read()
    if ret==False:
        continue
    gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(frame,1.3,5)     #1.3 scalling factor and 5 neighbor rectangles
    for face in faces:
        x,y,w,h = face
        #get the ROI
        offset=10     #paddinf of 10px
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100))
        
        out=knn(trainset,face_section.flatten())
        #display name on the screen and put a rectangle around it
        pred_name=names[int(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA) #check
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)     #check
    cv2.imshow("Frame",frame)
    
    key=cv2.waitKey(1) &0xFF
    if key==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

loaded amrit.npy
loaded sumit.npy
(73, 30000)
(73, 1)
(73, 30001)
